In [1]:
import pandas as pd 
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jatin\AppData\Roaming\nltk_data...


True

In [2]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection details
user = 'root'
password = 'jatin%409140'  # URL-encoded password ('@' becomes '%40')
host = 'localhost'
database = 'project'
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')
conn = engine.connect()

# Pull data from the table
table_name = 'customer_reviews'
query = f"select ReviewID,CustomerID,ProductID, ReviewDate,Rating,ReviewText from {table_name};"
data = pd.read_sql(query, conn)

# Display the data
print(data.head())
print("Success !")



   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  
0   Average  experience,  nothing  special.  
1            The  quality  is    top-notch.  
2   Five  stars  for  the  quick  delivery.  
3  Good  quality,  but  could  be  cheaper.  
4   Average  experience,  nothing  special.  
Success !


In [3]:
data.head()

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."


In [4]:
sia = SentimentIntensityAnalyzer()

In [5]:
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']

In [6]:
def categorize_sentiment(score,rating):
    if score>0.05:
        if rating>=4:
            return 'Positive'
        elif rating==3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score< -0.05:
        if rating<=2:
            return 'Negative'
        elif rating==3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive'
        
    else:
        if rating>=4:
            return 'Positive'
        elif rating<=2:
            return 'Negative'
        else:
            return 'Neutral'

In [7]:
def sentiment_bucket(score):
    if score>=0.5:
        return '0.5 to 1.0'
    elif 0.0<=score<0.5:
        return '0.0 to 0.49'
    elif -0.5<=score<0.0:
        return '-0.49 to 0.0'    
    else:
        return '-1.0 to -0.5' 


In [8]:
data['SentimentScore'] = data['ReviewText'].apply(calculate_sentiment)
data

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089
...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000


In [9]:
data['SentimentCategory'] = data.apply(
    lambda row: categorize_sentiment(row['SentimentScore'],row['Rating']),axis=1
)
data

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative
...,...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695,Mixed Negative
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089,Negative
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997,Positive
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000,Negative


In [10]:
data['SentimentBucket'] = data['SentimentScore'].apply(sentiment_bucket)
data

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive,0.0 to 0.49
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive,0.0 to 0.49
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive,0.0 to 0.49
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
...,...,...,...,...,...,...,...,...,...
1358,1359,28,4,2023-05-25,3,Not worth the money.,-0.1695,Mixed Negative,-0.49 to 0.0
1359,1360,58,12,2023-11-13,2,"Average experience, nothing special.",-0.3089,Negative,-0.49 to 0.0
1360,1361,96,15,2023-03-07,5,Customer support was very helpful.,0.6997,Positive,0.5 to 1.0
1361,1362,99,2,2025-12-03,1,Product did not meet my expectations.,0.0000,Negative,0.0 to 0.49


In [16]:
data.to_csv('customers_sentiment.csv',index=False )

In [15]:
df = pd.read_csv('customers_sentiment.csv')
user = 'root'
password = 'jatin%409140'  # URL-encoded password ('@' becomes '%40')
host = 'localhost'
database = 'project'
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')
conn = engine.connect()


In [16]:
table_name = 'customers_sentiment'
df.to_sql(name=table_name,con= conn,if_exists='replace',index=False)
print("Success !")

Success !
